In [43]:
# %load diffsolver.py
import numpy as np

class odeTime:
    
    def __init__(self, tFrom, tTo, tIntervals):
        self.tFrom = tFrom
        self.tTo = tTo
        self.tIntervals = tIntervals
        
class odeInput:
    
    #input
    #    time: odeTime
    #    initialValues: numpy matrix of dimension 1*n
    
    def __init__(self, time, initialValues):
        self.time = time
        self.initialValues = initialValues
    
    def length(self):
        return self.time.tIntervals
    
    def dimension(self):
        return self.initialValues.shape[0]

class odeOutput:
    
    def __init__(self, inputForOde):
        self.inputForOde = inputForOde
        self.results = np.zeros((inputForOde.length(), inputForOde.dimension()))
    
    def length(self):
        return self.inputForOde.length()
    
    def dimension(self):
        return self.inputForOde.dimension()
    

class odeSolver:
    
    def __init__(self, timeInfo, initialValues):
        inputForOde = odeInput(timeInfo, initialValues)
        self.inputForOde = inputForOde
        self.outputForOde = odeOutput(inputForOde)
        
    def equation(self, t, x):
        return x
    
    def solve(self):
        dt = (self.inputForOde.time.tTo - self.inputForOde.time.tFrom) / float(self.inputForOde.time.tIntervals)
        
        oldval = np.zeros((1,self.inputForOde.dimension()))
        newval = np.zeros((1,self.inputForOde.dimension()))
        
        # save initial data to output
        self.outputForOde.results[0] = self.inputForOde.initialValues
        
        # new value is set to the initial data at first
        newval = self.outputForOde.results[0]
        
        for i in range(1, self.inputForOde.length()):
            # move new values to old values
            oldval = newval
            
            tOld = self.inputForOde.time.tFrom + float(i-1) * dt
            
            k1 = self.equation(tOld, oldval)
            fTemp = 0.5 * dt * k1 + oldval
            
            k2 = self.equation(tOld + 0.5 * dt, fTemp)
            fTemp = 0.5 * dt * k2 + oldval
            
            k3 = self.equation(tOld + 0.5 * dt, fTemp)
            fTemp = 0.5 * dt * k3 + oldval
            
            k4= self.equation(tOld + 0.5 * dt, fTemp)
            fTemp = 0.5 * dt * k3 + oldval
            
            newval = oldval + (dt / 6.0) * (1.0 * k1 + 2.0 * k2 + 2.0 * k3 + 1.0 * k4)
            
            self.outputForOde.results[i] = newval
            
    def printResult(self):
        print("Solved ODE")
        print("Initial value: ", self.inputForOde.initialValues)
        print("Output:")
        dt = (self.inputForOde.time.tTo - self.inputForOde.time.tFrom) / float(self.inputForOde.time.tIntervals)
        for i in range(0, self.outputForOde.length()):
            t = self.inputForOde.time.tFrom + dt * float(i)
            print("t: ", t, "y: ", self.outputForOde.results[i])
        

In [47]:
timeInfo = odeTime(0.0, 1.1, 100)
initialValue = np.array([1.0])
solver = odeSolver(timeInfo, initialValue)

In [48]:
solver.solve()
solver.printResult()

Solved ODE
Initial value:  [ 1.]
Output:
t:  0.0 y:  [ 1.]
t:  0.011000000000000001 y:  [ 1.01105058]
t:  0.022000000000000002 y:  [ 1.02222328]
t:  0.033 y:  [ 1.03351945]
t:  0.044000000000000004 y:  [ 1.04494044]
t:  0.05500000000000001 y:  [ 1.05648764]
t:  0.066 y:  [ 1.06816244]
t:  0.07700000000000001 y:  [ 1.07996626]
t:  0.08800000000000001 y:  [ 1.09190052]
t:  0.099 y:  [ 1.10396666]
t:  0.11000000000000001 y:  [ 1.11616613]
t:  0.12100000000000001 y:  [ 1.12850042]
t:  0.132 y:  [ 1.14097101]
t:  0.14300000000000002 y:  [ 1.1535794]
t:  0.15400000000000003 y:  [ 1.16632713]
t:  0.165 y:  [ 1.17921572]
t:  0.17600000000000002 y:  [ 1.19224675]
t:  0.18700000000000003 y:  [ 1.20542177]
t:  0.198 y:  [ 1.21874238]
t:  0.20900000000000002 y:  [ 1.2322102]
t:  0.22000000000000003 y:  [ 1.24582684]
t:  0.231 y:  [ 1.25959395]
t:  0.24200000000000002 y:  [ 1.2735132]
t:  0.253 y:  [ 1.28758626]
t:  0.264 y:  [ 1.30181484]
t:  0.275 y:  [ 1.31620065]
t:  0.28600000000000003 y:  [ 1